In [1]:
import pandas as pd
import numpy as np
import requests as re
import bs4

# 2023

In [14]:
teams = ['stonehill-college', 'northern-colorado', 'utah', 'stonehill-college', 'utah', 'san-jose-state', 'utah-state', 
         'san-jose-state', 'utah-state', 'saint-mary-s', 'san-diego', 'portland-state', 'washington', 'texas-tech', 
         'oregon', 'loyola-marymount', 'boston-college', 'boston-college', 'boston-college', 'western-carolina', 'harvard',
        'california-baptist', 'syracuse', 'south-dakota', 'csu-bakersfield', 'csu-bakersfield', 'csu-bakersfield', 'uc-santa-barbara', 
         'uc-santa-barbara', 'uc-santa-barbara', 'hawai-i', 'hawai-i', 'hawai-i', 'ucla', 'long-beach-state', 'long-beach-state', 
         'long-beach-state', 'cal-state-fullerton', 'cal-state-fullerton', 'cal-state-fullerton', 'uc-davis', 'uc-davis', 
         'uc-davis', 'uc-riverside', 'uc-riverside', 'uc-riverside', 'csun', 'csun', 'csun', 'cal-poly', 'cal-poly', 'cal-poly']
nums = [9296, 9297, 9300, 9301, 9302, 9305, 9306, 9307, 9308, 9309, 9310, 9311, 9312, 9313, 9314, 9315, 9316, 9317, 9318, 9460,
       9320, 9322, 9323, 9324, 9326, 9327, 9328, 9329, 9330, 9331, 9332, 9333, 9334, 9304, 9335, 9402, 9403, 9404, 9405, 9406, 
        9408, 9409, 9410, 9411, 9412, 9413, 9414, 9415, 9416, 9417, 9419, 9420]
triton_innings23 = []
for i in range(52):
    play_by_play_link = 'https://ucsdtritons.com/sports/softball/stats/2023/' + teams[i] + '/boxscore/' + str(nums[i])
    play_soup = bs4.BeautifulSoup(re.get(play_by_play_link).content)
    play_by_play_html = play_soup.find('section', attrs={'id': 'play-by-play'})
    innings = play_by_play_html.find('div', attrs={'id': 'inning-all'}).find_all('div', attrs={'class': 'panel relative'})
    triton_innings23 += (list(filter(lambda x: x.find('caption').text.find('UC San Diego') > -1, innings)))
triton_innings23

[<div class="panel relative">
 <div class="row pad">
 <table class="sidearm-table play-by-play">
 <caption>UC San Diego - Bottom of 1st</caption>
 <thead>
 <tr>
 <th class="text-hide" scope="col">Play Description</th>
 <th class="text-center" scope="col">STO</th>
 <th class="text-center" scope="col">UCS</th>
 </tr>
 </thead>
 <tbody>
 <tr>
 <td class="text-italic">Williams, G to dh.</td>
 <td class="text-center">0</td>
 <td class="text-center">0</td>
 </tr>
 <tr>
 <td class="text-italic">/ to p for Williams, G.</td>
 <td class="text-center">0</td>
 <td class="text-center">0</td>
 </tr>
 <tr>
 <td>Pederson, D popped up to cf.</td>
 <td class="text-center">0</td>
 <td class="text-center">0</td>
 </tr>
 <tr>
 <td>Demarse, A reached on an error by 3b.</td>
 <td class="text-center">0</td>
 <td class="text-center">0</td>
 </tr>
 <tr>
 <td>Bosinger, K popped up to 2b.</td>
 <td class="text-center">0</td>
 <td class="text-center">0</td>
 </tr>
 <tr>
 <td>Williams, G popped up to cf.</td>
 <td 

Data to be extracted for each plate appearance:

- Name of hitter
- Number of outs
- Runners (keep track in 3 columns - either boolean or binary for each base)
- Score
- Tritons Away/Home
- Total Number of runs scored in inning
- Number of runs scored up to that plate appearance in inning

To be added later

- At least 1 run scored in inning (bool)
- wOBA of hitter
- Number of runs scored after plate appearance (including hitter currently up, so if there was 1 run already scored, and the current hitter hits a 2 run homer, the 2 runs count towards this data point)

Order of columns: batter, first, second, third, outs, away_score, home_score, Triton_side, total_runs_inn, runs_up_to_batter

Notes
- Parentheses in line means an at bat occurred and ended - start with this to indicate which block of code the line should be run through - some play by plays don't have the parentheses
- Line always starts with batter name in format Last, First initial.
- it will say if a runner scores, use the word 'scored' as indicator of how many runs were scored, not RBI (ex: Smith, J. singled through left side, RBI (2-1 BBK); Glascoe, L. advanced to second; Saunders, L. advanced to third; Kleszcz, B. scored.)
- semicolons indicate more happened on play, period at end of line indicates everything that happened has been stated

Outcomes
- All stated in the past tense (struck out, flied out, stole second, advanced to third, etc)
- strikeout
- flyout
- pop out (weird example: Sudyka, N. popped up to rf)
- lineout
- ground out
- foul out
- double play
- fielder's choice
- reach on error
- single
- double
- triple
- homerun
- walk
- intentional walk
- hit by pitch
- catcher's interference
- dropped 3rd strike

Non-at bat outcomes
- wild pitch
- stolen base
- fielder's indifference
- passed ball
- picked off
- caught stealing

In [3]:
def isbatter(txt):
    first_event = txt.split(';')[0]
    #if (first_event.find('advanced') > -1) and (first_event.find('on the throw') > -1):
        #return True
    if (first_event.find('stole') > -1) or ((first_event.find('advanced') > -1) and (first_event.split(' ')[2] == 'advanced')) or (first_event.find('scored') > -1) or (first_event.find('caught stealing') > -1) or (first_event.find('picked off') > -1):
        return False
    else:
        return True

In [4]:
batter_plays = []
non_batter_plays = []
for inns in triton_innings:
    pbp = inns.find_all('td')
    pbp_tags = [i for i in pbp if not i.attrs]
    pbp_text = [i.text for i in pbp_tags]
    for play in pbp_text:
        if isbatter(play):
            batter_plays.append(play)
        else:
            non_batter_plays.append(play)

In [5]:
inn1 = triton_innings[4]
pbp = inn1.find_all('td')
pbp_tags = [i for i in pbp if ((not i.attrs) or (i.attrs['class'][0] == 'text-italic'))]
pbp_text = [i.text for i in pbp_tags if ((not i.attrs) or (i.attrs['class'][0] == 'text-italic' and i.text.find('pinch ran') > -1))]
pbp_text[1].split(' ')[0][:-1]
pbp_text[1].split(' ')[-2][:-1]

'Richards'

In [94]:
names23 = ['Eng', 'Bosinger', 'Hassis', 'Aguirre', 'Reynoso', 'Mulligan', 'Hassis,M', 'Demarse', 'Pederson', 'Williams', 'Real', 
         'Richards', 'Heathcock', 'Pancino', 'Allen', 'Melendez', 'Hermosillo', 'Erickson', 'Dobson', 'Adams', 'Coberly', 'Renner']
def get_situations(triton_innings, names):
    pbp_data = []
    for inn in triton_innings:
        first = '_'
        second = '_'
        third = '_'
        outs = 0
        tot_runs = int(inn.find('dd').text)
        runs_so_far = 0
        #if inn.find('caption').text.find('Top') > -1:
            #home_away = 'Away'
        #else:
            #home_away = 'Home'
        #score = inn.find_all('td', attrs={'class': 'text-center'})[:2]
        #away_score = int(score[0].text)
        #home_score = int(score[1].text)
        pbp = inn.find_all('td')
        pbp_tags = [i for i in pbp if ((not i.attrs) or (i.attrs['class'][0] == 'text-italic'))]
        pbp_text = [i.text for i in pbp_tags if ((not i.attrs) or (i.attrs['class'][0] == 'text-italic' and i.text.find('pinch ran') > -1))]
        for play in pbp_text:
            if play.split(' ')[0].strip(',') not in names:
                continue
            if play.find('pinch ran') > -1:
                new_runner = play.split(' ')[0][:-1].strip(',')
                old_runner = play.split(' ')[-2][:-1].strip(',')
                if first == old_runner:
                    first = new_runner
                elif second == old_runner:
                    second = new_runner
                else:
                    third = new_runner
            elif isbatter(play):
                batter = play.split(' ')[0].strip(',')
                situation = [batter, first, second, third, outs, tot_runs, runs_so_far]
                pbp_data.append(situation)
                for event in play.split(';'):
                    player = event.split()[0].strip(',')
                    if event.find('reached') > -1:
                        if event.find('advanced') == -1:
                            first = player
                        elif event.find('second') > -1:
                            second = player
                        else:
                            third = player
                    elif (event.find('double play') > -1) or (event.find('out') > -1) or (event.find('popped up') > -1):
                        outs += 1
                        if first == player:
                            first = '_'
                        if second == player:
                            second == '_'
                        if third == player:
                            third == '_'
                    elif event.find('advanced') > -1:
                        if (event.find('second') > -1) and (event.find('third') == -1):
                            second = player
                            if first == player:
                                first = '_'
                            if third == player:
                                third = '_'
                        else:
                            third = player
                            if first == player:
                                first = '_'
                            if second == player:
                                second = '_'
                    elif (event.find('singled') > -1) or (event.find('walked') > -1) or (event.find('hit by pitch') > -1):
                        first = player
                    elif event.find('doubled') > -1:
                        second = player
                    elif event.find('tripled') > -1:
                        third = player
                    elif event.find('homered') > -1:
                        first = '_'
                        second = '_'
                        third = '_'
                        runs_so_far += 1
                        #if home_away == 'Away':
                            #away_score += 1
                        #else:
                            #home_score += 1
                    elif event.find('scored') > -1:
                        runs_so_far += 1
                        #if home_away == 'Away':
                            #away_score += 1
                        #else:
                            #home_score += 1
                        if first == player:
                            first = '_'
                        if second == player:
                            second = '_'
                        if third == player:
                            third = '_'
                    else:
                        continue
            else:
                for event in play.split(';'):
                    player = event.split(' ')[0].strip(',')
                    if event.find('out') > -1:
                        outs += 1
                        if first == player:
                            first = '_'
                        if second == player:
                            second = '_'
                        if third == player:
                            third = '_'
                    elif (event.find('advanced') > -1) or (event.find('stole') > -1):
                        if first == player:
                            first = '_'
                        if second == player:
                            second = '_'
                        if third == player:
                            third = '_'
                        if (event.find('second') > -1) and (event.find('third') == -1):
                            second = player
                        elif event.find('third') > -1:
                            third = player
                        else:
                            runs_so_far += 1
                            #if home_away == 'Away':
                                #away_score += 1
                            #else:
                                #home_score += 1
                    elif event.find('scored') > -1:
                        runs_so_far += 1
                        #if home_away == 'Away':
                            #away_score += 1
                        #else:
                            #home_score += 1
                        if first == player:
                            first = '_'
                        if second == player:
                            second = '_'
                        if third == player:
                            third = '_'
                    else:
                        continue

    situations = pd.DataFrame(data = pbp_data, columns = ['batter', 'first', 'second', 'third', 'outs', 'total_runs_inn', 'runs_up_to_batter'])
    return situations

situations = get_situations(triton_innings23, names23)
situations

,batter,first,second,third,outs,total_runs_inn,runs_up_to_batter
0,Pederson,_,_,_,0,0,0
1,Demarse,_,_,_,1,0,0
2,Bosinger,Demarse,_,_,1,0,0
3,Williams,Demarse,_,_,2,0,0
4,Reynoso,_,_,_,0,0,0
...,...,...,...,...,...,...,...
1521,Aguirre,_,_,_,2,0,0
1522,Reynoso,_,_,_,0,0,0
1523,Real,_,_,_,1,0,0
1524,Demarse,_,_,_,2,0,0


In [95]:
situations['runs_after_batter'] = situations['total_runs_inn'] - situations['runs_up_to_batter']
situations['at_least_1_scored'] = situations['runs_after_batter'] >= 1
#wOBA of hitter to be added
situations.iloc[:28]

,batter,first,second,third,outs,total_runs_inn,runs_up_to_batter,runs_after_batter,at_least_1_scored
0,Pederson,_,_,_,0,0,0,0,False
1,Demarse,_,_,_,1,0,0,0,False
2,Bosinger,Demarse,_,_,1,0,0,0,False
3,Williams,Demarse,_,_,2,0,0,0,False
4,Reynoso,_,_,_,0,0,0,0,False
5,Aguirre,_,_,_,1,0,0,0,False
6,Hassis,_,Aguirre,_,1,0,0,0,False
7,Richards,Hassis,Aguirre,_,1,0,0,0,False
8,Pancino,_,Hassis,Aguirre,2,0,0,0,False
9,Pederson,_,_,_,0,0,0,0,False


In [96]:
situations_base = situations.copy()
situations_base['first'] = situations_base['first'].apply(lambda x: '1' if x != '_' else x)
situations_base['second'] = situations_base['second'].apply(lambda x: '2' if x != '_' else x)
situations_base['third'] = situations_base['third'].apply(lambda x: '3' if x != '_'else x)
situations_base['base'] = situations_base['first'] + ' ' + situations_base['second'] + ' ' + situations_base['third']
situations_base[:28]

,batter,first,second,third,outs,total_runs_inn,runs_up_to_batter,runs_after_batter,at_least_1_scored,base
0,Pederson,_,_,_,0,0,0,0,False,_ _ _
1,Demarse,_,_,_,1,0,0,0,False,_ _ _
2,Bosinger,1,_,_,1,0,0,0,False,1 _ _
3,Williams,1,_,_,2,0,0,0,False,1 _ _
4,Reynoso,_,_,_,0,0,0,0,False,_ _ _
5,Aguirre,_,_,_,1,0,0,0,False,_ _ _
6,Hassis,_,2,_,1,0,0,0,False,_ 2 _
7,Richards,1,2,_,1,0,0,0,False,1 2 _
8,Pancino,_,2,3,2,0,0,0,False,_ 2 3
9,Pederson,_,_,_,0,0,0,0,False,_ _ _


In [87]:
situations_base[(situations_base['outs'] == 0) & (situations_base['base'] == '1 2 3')]
situations_base[838:845]

,batter,first,second,third,outs,total_runs_inn,runs_up_to_batter,runs_after_batter,at_least_1_scored,base
838,Mulligan,1,2,3,0,0,0,0,False,1 2 3
839,Eng,1,2,3,1,0,0,0,False,1 2 3
840,Pederson,_,_,_,0,0,0,0,False,_ _ _
841,Aguirre,_,_,_,1,0,0,0,False,_ _ _
842,Real,_,_,_,2,0,0,0,False,_ _ _
843,Reynoso,1,_,_,2,0,0,0,False,1 _ _
844,Bosinger,_,_,_,0,0,0,0,False,_ _ _


In [88]:
# create run expectancy matrix and run probability matrix using pivot tables

In [97]:
re_matrix = pd.pivot_table(data = situations_base, values = ['runs_after_batter'], index = ['base'], columns = ['outs'], aggfunc = 'mean')
re_matrix

runs_after_batter                    
outs                  0         1         2
base                                       
1 2 3          0.500000  0.863636  0.590909
1 2 _          1.000000  0.480769  0.222222
1 _ 3          1.000000  0.818182  0.350000
1 _ _          0.531915  0.255102  0.150538
_ 2 3          0.833333  1.115385  0.310345
_ 2 _          1.076923  0.432836  0.310345
_ _ 3          1.250000  0.833333  0.323529
_ _ _          0.335180  0.182573  0.122581

In [98]:
rp_matrix = pd.pivot_table(data = situations_base, values = ['at_least_1_scored'], index = ['base'], columns = ['outs'], aggfunc = 'mean')
rp_matrix

at_least_1_scored                    
outs                  0         1         2
base                                       
1 2 3          0.500000  0.545455  0.363636
1 2 _          0.500000  0.269231  0.185185
1 _ 3          0.600000  0.545455  0.200000
1 _ _          0.319149  0.142857  0.107527
_ 2 3          0.500000  0.538462  0.172414
_ 2 _          0.653846  0.298507  0.241379
_ _ 3          0.750000  0.625000  0.205882
_ _ _          0.210526  0.116183  0.077419

Risk x runs to gain y runs - find odds to make it work the risk like poker

Example:
Risk 0.33 runs to gain 0.66 runs - 2/1 ratio means if there are >50% odds for success, the play produces a positive expected value

Another option (choices with more than 2 outcomes) - total expected risk/reward based on percentage of each outcome occurring

From [article](https://www.pitcherlist.com/adjusting-for-the-current-run-expectancy-matrix/): break even point = |failure - current| / (|failure - current| + |success - current|)

In [99]:
def calc_be_point(current, success, failure, matrix = 're'):
    if matrix == 're':
        s_runs = re_matrix.loc[success['base']].loc['runs_after_batter'].loc[success['outs']]
        c_runs = re_matrix.loc[current['base']].loc['runs_after_batter'].loc[current['outs']]
        if failure['outs'] != 3:
            f_runs = re_matrix.loc[failure['base']].loc['runs_after_batter'].loc[failure['outs']]
        else:
            f_runs = 0
    else:
        s_runs = rp_matrix.loc[success['base']].loc['at_least_1_scored'].loc[success['outs']]
        c_runs = rp_matrix.loc[current['base']].loc['at_least_1_scored'].loc[current['outs']]
        if failure['outs'] != 3:
            f_runs = rp_matrix.loc[failure['base']].loc['at_least_1_scored'].loc[failure['outs']]
        else:
            f_runs = 0
    risk = c_runs - f_runs
    reward = s_runs - c_runs
    if reward < 0:
        return 'negative expected value play'
    break_even = risk / (risk + reward)
    return break_even
    
calc_be_point({'base': '_ 2 _', 'outs': 2}, {'base': '_ _ 3', 'outs': 2}, {'base': 'inning over', 'outs': 3}, matrix = 're')

0.9592476489028213

# 2022

In [21]:
teams22 = ['nevada', 'ucla', 'oregon', 'loyola-marymount', 'oklahoma', 'saint-mary-s', 'sacramento-state', 'utah', 'saint-mary-s', 
        'sacramento-state', 'tennessee', 'seattle-u', 'california-baptist', 'southern-utah', 'oregon-state', 'yale', 'byu',
        'boston-u', 'utah-state', 'cal', 'princeton', 'princeton', 'california-baptist', 'california-baptist', 'csu-bakersfield', 
        'csu-bakersfield', 'csu-bakersfield', 'uc-santa-barbara', 'uc-santa-barbara', 'uc-santa-barbara', 'university-of-san-diego',
        'hawai-i', 'hawai-i', 'hawai-i', 'long-beach-state', 'long-beach-state', 'long-beach-state', 'cal-state-fullerton', 
        'cal-state-fullerton', 'cal-state-fullerton', 'san-diego-state', 'uc-davis', 'uc-davis', 'uc-davis', 'uc-riverside', 
        'uc-riverside', 'uc-riverside', 'csun', 'csun', 'csun', 'cal-poly', 'cal-poly', 'cal-poly', 'rutgers', 'baylor', 'tarleton-state']
nums22 = [8731, 8732, 8738, 8739, 8740, 8735, 8827, 8736, 8828, 8737] + list(range(8741, 8751)) + [8733, 8734, 8751, 8845] + list(range(8752, 8758)) + [8920] + list(range(8758, 8767)) + [8846] + list(range(8767, 8779)) + [8933, 8934, 8935]
triton_innings22 = []
for i in range(56):
    play_by_play_link = 'https://ucsdtritons.com/sports/baseball/stats/2022/' + teams22[i] + '/boxscore/' + str(nums22[i])
    play_soup = bs4.BeautifulSoup(re.get(play_by_play_link).content)
    play_by_play_html = play_soup.find('section', attrs={'id': 'play-by-play'})
    innings = play_by_play_html.find('div', attrs={'id': 'inning-all'}).find_all('div', attrs={'class': 'panel relative'})
    triton_innings22 += (list(filter(lambda x: x.find('caption').text.find('UC San Diego') > -1, innings)))
triton_innings22

[<div class="panel relative">
 <div class="row pad">
 <table class="sidearm-table play-by-play">
 <caption>UC San Diego - Bottom of 1st</caption>
 <thead>
 <tr>
 <th class="text-hide" scope="col">Play Description</th>
 <th class="text-center" scope="col">ORE</th>
 <th class="text-center" scope="col">UCS</th>
 </tr>
 </thead>
 <tbody>
 <tr>
 <td>Real, S grounded out to 3b (1-2 KSBF).</td>
 <td class="text-center">2</td>
 <td class="text-center">0</td>
 </tr>
 <tr>
 <td>Bosinger, K flied out to cf (3-2 FBBKBFF).</td>
 <td class="text-center">2</td>
 <td class="text-center">0</td>
 </tr>
 <tr>
 <td>Aguirre, B struck out swinging (0-2 FFS).</td>
 <td class="text-center">2</td>
 <td class="text-center">0</td>
 </tr>
 </tbody>
 </table>
 <dl class="special-stats emphasize inline text-right">
 <dt>Runs</dt>
 <dd>0</dd>
 <dt>Hits</dt>
 <dd>0</dd>
 <dt>Errors</dt>
 <dd>0</dd>
 <dt>Left On Base</dt>
 <dd>0</dd>
 </dl>
 </div>
 </div>,
 <div class="panel relative">
 <div class="row pad">
 <table 

In [100]:
names22 = ['Lavrov', 'Aguirre', 'Bosinger', 'Pederson', 'Demarse', 'Reynoso', 'Hassis', 'Real', 'Williams', 'Renner', 'Richards',
        'Eng', 'Erickson', 'Pancino', 'Melendez']
situations22 = get_situations(triton_innings22, names22)
situations22['runs_after_batter'] = situations22['total_runs_inn'] - situations22['runs_up_to_batter']
situations22['at_least_1_scored'] = situations22['runs_after_batter'] >= 1
situations_base22 = situations22.copy()
situations_base22['first'] = situations_base22['first'].apply(lambda x: '1' if x != '_' else x)
situations_base22['second'] = situations_base22['second'].apply(lambda x: '2' if x != '_' else x)
situations_base22['third'] = situations_base22['third'].apply(lambda x: '3' if x != '_'else x)
situations_base22['base'] = situations_base22['first'] + ' ' + situations_base22['second'] + ' ' + situations_base22['third']
situations_base22

,batter,first,second,third,outs,total_runs_inn,runs_up_to_batter,runs_after_batter,at_least_1_scored,base
0,Real,_,_,_,0,0,0,0,False,_ _ _
1,Bosinger,_,_,_,1,0,0,0,False,_ _ _
2,Aguirre,_,_,_,2,0,0,0,False,_ _ _
3,Williams,_,_,_,0,0,0,0,False,_ _ _
4,Lavrov,_,2,_,0,0,0,0,False,_ 2 _
...,...,...,...,...,...,...,...,...,...,...
1593,Aguirre,1,_,_,2,0,0,0,False,1 _ _
1594,Lavrov,1,2,_,2,0,0,0,False,1 2 _
1595,Bosinger,_,_,_,0,0,0,0,False,_ _ _
1596,Reynoso,_,_,_,1,0,0,0,False,_ _ _


# 2021

In [25]:
teams21 = ['grand-canyon', 'grand-canyon', 'utah-state', 'utah-state', 'ucla', 'san-diego', 'san-diego', 'uc-santa-barbara', 
        'uc-santa-barbara', 'uc-santa-barbara', 'uc-santa-barbara', 'hawai-i', 'hawai-i', 'hawai-i', 'hawai-i',
        'long-beach-state', 'long-beach-state', 'long-beach-state', 'long-beach-state', 'cal-state-fullerton', 'cal-state-fullerton', 
        'cal-state-fullerton', 'cal-state-fullerton', 'uc-davis', 'uc-davis', 'uc-davis', 'uc-riverside', 'uc-riverside', 
        'uc-riverside', 'csun', 'csun', 'csun', 'cal-poly', 'cal-poly', 'cal-poly']
nums21 = list(range(8190, 8197)) + [8238, 8239, 8242, 8243, 8240, 8241] + list(range(8244, 8250)) + [8262, 8264, 8265, 8266, 8268, 8269, 8271, 8272, 8274, 8276, 8277, 8279, 8280, 8281, 8282, 8283]
triton_innings21 = []
for i in range(35):
    play_by_play_link = 'https://ucsdtritons.com/sports/baseball/stats/2021/' + teams21[i] + '/boxscore/' + str(nums21[i])
    play_soup = bs4.BeautifulSoup(re.get(play_by_play_link).content)
    play_by_play_html = play_soup.find('section', attrs={'id': 'play-by-play'})
    innings = play_by_play_html.find('div', attrs={'id': 'inning-all'}).find_all('div', attrs={'class': 'panel relative'})
    triton_innings21 += (list(filter(lambda x: x.find('caption').text.find('UC San Diego') > -1, innings)))
triton_innings21

[<div class="panel relative">
 <div class="row pad">
 <table class="sidearm-table play-by-play">
 <caption>UC San Diego - Bottom of 1st</caption>
 <thead>
 <tr>
 <th class="text-hide" scope="col">Play Description</th>
 <th class="text-center" scope="col">GCU</th>
 <th class="text-center" scope="col">UCS</th>
 </tr>
 </thead>
 <tbody>
 <tr>
 <td>Real flied out to rf (0-1 K).</td>
 <td class="text-center">0</td>
 <td class="text-center">0</td>
 </tr>
 <tr>
 <td>Pederson struck out swinging (2-2 BKBKF).</td>
 <td class="text-center">0</td>
 <td class="text-center">0</td>
 </tr>
 <tr>
 <td>Bosinger doubled to right field (2-0 BB).</td>
 <td class="text-center">0</td>
 <td class="text-center">0</td>
 </tr>
 <tr>
 <td>Harrington lined out to cf (1-1 KB).</td>
 <td class="text-center">0</td>
 <td class="text-center">0</td>
 </tr>
 </tbody>
 </table>
 <dl class="special-stats emphasize inline text-right">
 <dt>Runs</dt>
 <dd>0</dd>
 <dt>Hits</dt>
 <dd>1</dd>
 <dt>Errors</dt>
 <dd>0</dd>
 <dt>L

In [105]:
def get_situations21(triton_innings, names):
    pbp_data = []
    for inn in triton_innings:
        first = '_'
        second = '_'
        third = '_'
        outs = 0
        tot_runs = int(inn.find('dd').text)
        runs_so_far = 0
        #if inn.find('caption').text.find('Top') > -1:
            #home_away = 'Away'
        #else:
            #home_away = 'Home'
        #score = inn.find_all('td', attrs={'class': 'text-center'})[:2]
        #away_score = int(score[0].text)
        #home_score = int(score[1].text)
        pbp = inn.find_all('td')
        pbp_tags = [i for i in pbp if ((not i.attrs) or (i.attrs['class'][0] == 'text-italic'))]
        pbp_text = [i.text for i in pbp_tags if ((not i.attrs) or (i.attrs['class'][0] == 'text-italic' and i.text.find('pinch ran') > -1))]
        for play in pbp_text:
            if play.split(' ')[0] not in names:
                continue
            if play.find('pinch ran') > -1:
                new_runner = play.split(' ')[0]
                old_runner = play.split(' ')[-1].strip('.')
                if first == old_runner:
                    first = new_runner
                elif second == old_runner:
                    second = new_runner
                else:
                    third = new_runner
            elif isbatter(play):
                batter = play.split(' ')[0]
                situation = [batter, first, second, third, outs, tot_runs, runs_so_far]
                pbp_data.append(situation)
                for event in play.split(';'):
                    player = event.split()[0]
                    if event.find('reached') > -1:
                        if event.find('advanced') == -1:
                            first = player
                        elif event.find('second') > -1:
                            second = player
                        else:
                            third = player
                    elif (event.find('double play') > -1) or (event.find('out') > -1) or (event.find('popped up') > -1):
                        outs += 1
                        if first == player:
                            first = '_'
                        if second == player:
                            second == '_'
                        if third == player:
                            third == '_'
                    elif event.find('advanced') > -1:
                        if (event.find('second') > -1) and (event.find('third') == -1):
                            second = player
                            if first == player:
                                first = '_'
                            if third == player:
                                third = '_'
                        else:
                            third = player
                            if first == player:
                                first = '_'
                            if second == player:
                                second = '_'
                    elif (event.find('singled') > -1) or (event.find('walked') > -1) or (event.find('hit by pitch') > -1):
                        first = player
                    elif event.find('doubled') > -1:
                        second = player
                    elif event.find('tripled') > -1:
                        third = player
                    elif event.find('homered') > -1:
                        first = '_'
                        second = '_'
                        third = '_'
                        runs_so_far += 1
                        #if home_away == 'Away':
                            #away_score += 1
                        #else:
                            #home_score += 1
                    elif event.find('scored') > -1:
                        runs_so_far += 1
                        #if home_away == 'Away':
                            #away_score += 1
                        #else:
                            #home_score += 1
                        if first == player:
                            first = '_'
                        if second == player:
                            second = '_'
                        if third == player:
                            third = '_'
                    else:
                        continue
            else:
                for event in play.split(';'):
                    player = event.split(' ')[0].strip(',')
                    if event.find('out') > -1:
                        outs += 1
                        if first == player:
                            first = '_'
                        if second == player:
                            second = '_'
                        if third == player:
                            third = '_'
                    elif (event.find('advanced') > -1) or (event.find('stole') > -1):
                        if first == player:
                            first = '_'
                        if second == player:
                            second = '_'
                        if third == player:
                            third = '_'
                        if (event.find('second') > -1) and (event.find('third') == -1):
                            second = player
                        elif event.find('third') > -1:
                            third = player
                        else:
                            runs_so_far += 1
                            #if home_away == 'Away':
                                #away_score += 1
                            #else:
                                #home_score += 1
                    elif event.find('scored') > -1:
                        runs_so_far += 1
                        #if home_away == 'Away':
                            #away_score += 1
                        #else:
                            #home_score += 1
                        if first == player:
                            first = '_'
                        if second == player:
                            second = '_'
                        if third == player:
                            third = '_'
                    else:
                        continue

    situations = pd.DataFrame(data = pbp_data, columns = ['batter', 'first', 'second', 'third', 'outs', 'total_runs_inn', 'runs_up_to_batter'])
    return situations

In [106]:
names21 = ['Aguirre', 'Bosinger', 'Harrington', 'Kazakoff', 'Pederson', 'Lavrov', 'Real', 'Renner', 'Nixon', 'Tirado', 
          'Williams', 'Reynoso', 'Lane', 'Pancino', 'Kyro', 'Melendez', 'Richards', 'Dragoo', 'Wampler']
situations21 = get_situations21(triton_innings21, names21)
situations21['runs_after_batter'] = situations21['total_runs_inn'] - situations21['runs_up_to_batter']
situations21['at_least_1_scored'] = situations21['runs_after_batter'] >= 1
situations_base21 = situations21.copy()
situations_base21['first'] = situations_base21['first'].apply(lambda x: '1' if x != '_' else x)
situations_base21['second'] = situations_base21['second'].apply(lambda x: '2' if x != '_' else x)
situations_base21['third'] = situations_base21['third'].apply(lambda x: '3' if x != '_'else x)
situations_base21['base'] = situations_base21['first'] + ' ' + situations_base21['second'] + ' ' + situations_base21['third']
situations_base21[:40]

,batter,first,second,third,outs,total_runs_inn,runs_up_to_batter,runs_after_batter,at_least_1_scored,base
0,Real,_,_,_,0,0,0,0,False,_ _ _
1,Pederson,_,_,_,1,0,0,0,False,_ _ _
2,Bosinger,_,_,_,2,0,0,0,False,_ _ _
3,Harrington,_,2,_,2,0,0,0,False,_ 2 _
4,Aguirre,_,_,_,0,1,0,1,True,_ _ _
5,Lavrov,_,_,_,1,1,0,1,True,_ _ _
6,Pancino,_,2,_,1,1,0,1,True,_ 2 _
7,Kyro,_,_,3,2,1,0,1,True,_ _ 3
8,Kazakoff,1,_,_,2,1,1,0,False,1 _ _
9,Real,_,_,_,0,0,0,0,False,_ _ _


In [110]:
sits2123 = pd.concat([situations_base, situations_base22, situations_base21])
sits2123.to_csv(path_or_buf = 'softball_atbats_2021-2023.csv', index = False)
sits2123

,batter,first,second,third,outs,total_runs_inn,runs_up_to_batter,runs_after_batter,at_least_1_scored,base
0,Pederson,_,_,_,0,0,0,0,False,_ _ _
1,Demarse,_,_,_,1,0,0,0,False,_ _ _
2,Bosinger,1,_,_,1,0,0,0,False,1 _ _
3,Williams,1,_,_,2,0,0,0,False,1 _ _
4,Reynoso,_,_,_,0,0,0,0,False,_ _ _
...,...,...,...,...,...,...,...,...,...,...
1116,Aguirre,1,2,3,2,4,0,4,True,1 2 3
1117,Williams,_,2,3,2,4,2,2,True,_ 2 3
1118,Harrington,1,_,3,2,4,3,1,True,1 _ 3
1119,Lavrov,1,2,_,2,4,4,0,False,1 2 _


In [111]:
re_matrix_all = pd.pivot_table(data = sits2123, values = ['runs_after_batter'], index = ['base'], columns = ['outs'], aggfunc = 'mean')
re_matrix_all

runs_after_batter                    
outs                  0         1         2
base                                       
1 2 3          1.812500  1.500000  0.573529
1 2 _          1.555556  0.795276  0.307692
1 _ 3          1.642857  1.097561  0.500000
1 _ _          0.820896  0.418440  0.158301
_ 2 3          1.260870  1.255556  0.477273
_ 2 _          1.173333  0.785714  0.236364
_ _ 3          1.000000  0.985294  0.483146
_ _ _          0.486125  0.248013  0.100000

In [112]:
rp_matrix_all = pd.pivot_table(data = sits2123, values = ['at_least_1_scored'], index = ['base'], columns = ['outs'], aggfunc = 'mean')
rp_matrix_all

at_least_1_scored                    
outs                  0         1         2
base                                       
1 2 3          0.875000  0.702703  0.308824
1 2 _          0.611111  0.362205  0.195804
1 _ 3          0.785714  0.634146  0.258621
1 _ _          0.388060  0.212766  0.108108
_ 2 3          0.695652  0.600000  0.284091
_ 2 _          0.666667  0.403061  0.175758
_ _ 3          0.733333  0.705882  0.269663
_ _ _          0.268243  0.149444  0.069767